In [1]:
import numpy as np
import itertools
import operator
import copy
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


In [2]:
### target_files

cn_target_file='./very_new_cn_all'
cn_source_file='./new_cn_source'
vm_file='./very_new_vms'
cn_migr='./very_new_cn_migr'
k_vms = './very_new_vm_krasnoyarsk'
az1_file=  './z1'
az2_file = './z2'
az3_file = './z3'




In [3]:
with open(az1_file, 'r') as f:
    az1_l = f.readlines()
    az1_l = [i.replace('\t1\n','') for i in az1_l]
f.close()
with open(az2_file, 'r') as f:
    az2_l= f.readlines()
    az2_l = [i.replace('\t2\n','') for i in az2_l]
f.close()
with open(az3_file, 'r') as f:
    az3_l = f.readlines()
    az3_l = [i.replace('\t3\n','') for i in  az3_l]
f.close()

In [4]:
#hpd
#az2_l

In [5]:
### res_s + examples
cpu_per_cn = 56
ram_per_cn = 516990
#ram_per_cn = 493040
test_machine = 'site-liferay-10'
test_hypervisor = 'cn-d720.egron.tnx'
test_hypervisor_alter = 'cn-dc6c.egron.tnx'
cpu_coef = 0.00000002
ram_coef = 1.5

In [6]:
with open(vm_file, 'r') as f:
    vml = f.readlines()
    vml = [i.replace('\n','').split(',') for i in vml]
    
f.close()
with open(cn_target_file, 'r') as f:
    hpl = f.readlines()
    hpl = [i.replace('\n','').split(',') for i in hpl]

with open(cn_migr, 'r') as f:
    mhpl = f.readlines()
    mhpl = [i.replace('\n','').split(',')[0] for i in mhpl]

with open(k_vms, 'r') as f:
    kvml = f.readlines()
    kvml = [i.replace('\n','') for i in kvml]
    
    
    
f.close()

In [7]:
#hpl
#kvml

In [8]:
def vm_dict_create(sl):
    'создаем словарь виртуальных машин'
    
    rd = {}
    for i in sl:
        if i[1]=='':
            crit='E'
        else:
            crit=i[1]
        if i[0] in kvml:
            kr = 1
        else:
            kr=0
        rd[i[0]] = {'name': i[0],\
                 'cpu' : int(i[2]),\
                 'ram' : int(i[3]),\
                 'cn_name':i[4],\
                 'crit_rate':crit,\
                 'krasnoyarsk':kr


        }
    return rd, copy.deepcopy(rd)


def hyper_dict_create(c,r,hl,sl):
    'создаем  словарь гипервизоров'
    rd= {}
    for i in hl:
        h_vm_list = [s[0] for s in sl if s[4] == i[0]]
        #print(i[1])
        if i[1] in az1_l:
            az = 1
        elif i[1] in az2_l:
            az = 2
        elif i[1] in az3_l:
            az = 3
        if i[0] in mhpl:
            migr = 1
        else:
            migr = 0
        rd[i[0]] = { 'name' : i[0],\
                    'label': i[1],\
                         'cpu'  : c,\
                         'ram'  : r,\
                         'vms'  : h_vm_list,\
                         'vms_count' : len(h_vm_list),\
                         'az' : az,\
                         'migr' : migr


            }
    return rd

def hyper_dict_resources_create(hd,vd):
    rd = copy.deepcopy(hd)
    for i in rd.keys():
        #print([vd[j]['ram'] for j in rd[i]['vms']] )
        sum_ram_perc = (rd[i]['ram'] - sum([vd[j]['ram'] for j in rd[i]['vms']]))/rd[i]['ram'] 
        sum_cpu_perc = (rd[i]['cpu'] - sum([vd[j]['cpu'] for j in rd[i]['vms']]))/rd[i]['cpu']
        rd[i]['perc_cpu'] = sum_cpu_perc
        rd[i]['perc_ram'] = sum_ram_perc
        rd[i]['free_cpu'] = (rd[i]['cpu'] - sum([vd[j]['cpu'] for j in rd[i]['vms']]))
        rd[i]['free_ram'] = (rd[i]['ram'] - sum([vd[j]['ram'] for j in rd[i]['vms']]))
    return rd, copy.deepcopy(rd)

In [9]:
def count_over (h,v):
    vm_vcpu_sum = sum([v[i]['cpu'] for i in v.keys()])
    vm_ram_sum = sum([v[i]['ram'] for i in v.keys()])
    hyper_vcpu_sum = sum([h[i]['cpu'] for i in h.keys()])
    hyper_ram_sum = sum([h[i]['ram'] for i in h.keys()])
    
    return [vm_vcpu_sum, vm_ram_sum], [hyper_vcpu_sum,hyper_ram_sum],\
        [vm_vcpu_sum / hyper_vcpu_sum, vm_ram_sum/hyper_ram_sum] 

In [10]:
a1 = [hpd[i]['az'] for i in hpd.keys() if hpd[i]['az'] == 1]
a2 = [hpd[i]['az'] for i in hpd.keys() if hpd[i]['az'] == 2]
a3 = [hpd[i]['az'] for i in hpd.keys() if hpd[i]['az'] == 3]
migr = [hpd[i]['migr'] for i in hpd.keys() if hpd[i]['migr'] == 1]
len(a1),len(a2),len(a3), len(migr)

NameError: name 'hpd' is not defined

In [11]:
vmd, vmd_in_use = vm_dict_create(sl=vml)
hpd= hyper_dict_create(cpu_per_cn,ram_per_cn,hpl,vml)
hpd_res, hpd_res_in_use = hyper_dict_resources_create(hpd,vmd)
vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hpd_res,vmd)

In [352]:
#vmd[test_machine],hpd[test_hypervisor],hpd_res[test_hypervisor],hpd_res_in_use[test_hypervisor]

In [353]:
l = [hpd_res[h]['perc_ram'] for h in hpd_res.keys()]
m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]



print("Всего машин:", len(vml))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
print("Всего ВУ:", len(hpl))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))


Всего машин: 1654
Необходимо машинам vCPU : 12118 , RAM : 34728472 MB
На одном ВУ CPU : 56 , RAM : 516990 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 39291240 MB
Суммарная подписка по CPU : 2.8472744360902253 , RAM : 0.8838731483149934
Забитость ВУ по RAM
Самый забитый : 1.338950463258477
Наименее забитый : 0.3802936227006325
Средняя загруженность : 0.8849156198684491

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 4.107142857142858
Наименее забитый : 1.3571428571428572
Средняя загруженность : 2.8500939849624056


### Заполняем словарь hpd_in_use информацией о заполненности

In [354]:
#hpd_res_in_use[test_hypervisor_alter]['vms']

In [355]:
#vmd

In [356]:
s = [v for v in vmd.keys() if vmd[v]['krasnoyarsk'] == 1]
ss =[v for v in vmd.keys() if vmd[v]['krasnoyarsk'] == 0]

In [357]:
len(s), len(ss)

(488, 1164)

In [358]:
vmd['ppoz-db-fs-18']

{'cn_name': 'cn-e828.egron.tnx',
 'cpu': 8,
 'crit_rate': 'E',
 'krasnoyarsk': 1,
 'name': 'ppoz-db-fs-18',
 'ram': 32768}

### Распределение в случае разогретого словаря



In [359]:
def start_iter_distribution(iterations=10,ram_coef=ram_coef,cpu_coef=cpu_coef):
    rd={}
    iter_num = 0
    hd = copy.deepcopy(hpd_res_in_use)
    vd = copy.deepcopy(vmd_in_use)
    while iter_num < iterations:
        rd[iter_num] = {}
        #ram_list = [[i,hd[i]['perc_ram']]for i in hd.keys()]
        #worst_cn = sorted(ram_list, key = lambda x: float(x[1]))[:1][0][0]
        #### Посчитаем score для всех с учетом двух значений 
        scored_list = [[i,((ram_coef * hpd_res_in_use[i]['perc_ram'] + cpu_coef * hpd_res_in_use[i]['perc_cpu']))]\
                       for i in hpd_res_in_use.keys()]
        worst_cn = sorted(scored_list, key = lambda x: float(x[1]))[:1][0][0]
        #print(worst_cn)
        mean_memory =  mean([vmd[j]['ram'] for j in hd[worst_cn]['vms']])
        #cand_list = [[vmd[j]['ram'],vmd[j]['cpu'], j ]for j in hd[worst_cn]['vms'] if vmd[j]['ram'] <= mean_memory ]
        cand_list = [[vmd[j]['ram'],vmd[j]['cpu'], j ]for j in hd[worst_cn]['vms'] if vmd[j]['ram'] <= mean_memory ]
        cand = sorted(cand_list, key=operator.itemgetter(0, 1))[:1]
        #print(iter_num,worst_cn,cand[0],cand_list[:3])
        ##print(iter_num,worst_cn,sorted(scored_list, key = lambda x: float(x[1]))[:3])
       # print(iter_num,worst_cn,cand[0],cand_list[:2])
        vm_to_migrate = cand[0][2]
        vm, ex, new, bc = migrate_vm(v=vm_to_migrate,hd=hd,vd=vd)
        
        rd[iter_num]['from']=ex
        rd[iter_num]['to']=new
        rd[iter_num]['vm']=vm
        rd[iter_num]['cand_hyp_from']=sorted(scored_list, key = lambda x: float(x[1]))[:3]
        rd[iter_num]['cand_vm']=sorted(cand_list,reverse=False)[:2]
        rd[iter_num]['cand_hyp_to']= bc
        ##print(iter_num,worst_cn,new,cand[0],sorted(cand_list,reverse=False)[:2])
        iter_num = iter_num + 1
        #print(iter_num,worst_cn, cand)
        #print(iter_num,sorted(ram_list, key = lambda x: int(x[1]))[:3])
        #print(iter_num,vm_to_migrate,vmd[vm_to_migrate]['cn_name'],vmd_in_use[vm_to_migrate]['cn_name'])

    return rd
        
    
    
def migrate_vm(v,hd,vd):
    ex_ = vd[v]['cn_name']
    best_cand = find_best_hid_by_score(v,hd)
    cn_best_cand = best_cand[0][0]
    #print('\t',v,vmd[v]['cn_name'],vmd_in_use[v]['cn_name'],vd[v]['cn_name'])
    calculate_after_loc(exh=ex_,hd=hd,vd=vd,h=cn_best_cand,v=v)
    #print('\t',v,vmd[v]['cn_name'],vmd_in_use[v]['cn_name'],vd[v]['cn_name'])
    return v, ex_, cn_best_cand, best_cand

def find_best_hid_by_score(v,hd):
    rd={}
    for h in hd.keys(): 
        if h == vmd_in_use[v]['cn_name']:
            continue
        #free_ram = hd[h]['free_ram']
        #free_cpu = hd[h]['free_cpu']
        free_ram = hd[h]['perc_ram']
        free_cpu = hd[h]['perc_cpu']
        if free_ram < 0:
            continue
        score = count_score(free_cpu,free_ram)
        rd[h]={#'mply': score_mply,\
                              # 'score_az':score_az,\
                              # 'score_hyp':score_hyp, 
                               'score': score,\
                               'id' : h,\
                               'ram': hd[h]['free_ram'],\
                               'cpu': hd[h]['free_cpu'],\
                               'vm_count' : hd[h]['vms']
                              }
    ##### исходя из того, что словарь приходит разогретым - мы можем позволить себе выбрать просто первого кандидата
    scoring_list = [[i, rd[i]['score'], rd[i]['ram'], rd[i]['cpu']] for i in rd.keys() ]
    best_buddy = sorted(scoring_list, key = lambda x: float(x[1]), reverse=True)[:5]
    return best_buddy

        
def count_score(pc,pr):
    
    res = (cpu_coef * pc + ram_coef * pr)/2

    ### score = 1.2 * az * 2.5 * hz * (1 - mw)
    ### Учитываем ресурсы
    score = res
    #print(res,az,hz,mw,score)
    return score
    '''    
def calculate_after_loc(v,h,exh,hd,vd):
    ### Изменения во временных словарях после локации
    ###  # заполняем значения для выбранного кандидата
    ## отнимаем абсолютные и относительные значения ресурсов
    hd[h]['free_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) 
    hd[h]['perc_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) / cpu_per_cn
    hd[h]['free_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) 
    hd[h]['perc_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[h]['vms_count'] = (hd[h]['vms_count'] + 1)
    ## обновляем список машин на ВУ
    hd[h]['vms'].append(v)
    ### # заполняем значения для ex-ВУ
    ## прибавляем ресурсы 
    hd[exh]['free_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) 
    hd[exh]['perc_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) / cpu_per_cn
    hd[exh]['free_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) 
    hd[exh]['perc_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[exh]['vms_count'] = (hd[exh]['vms_count'] - 1)
    ## обновляем список машин на ВУ
    hd[exh]['vms'].remove(v)
    vd[v]['cn_name'] = h
    
'''
def calculate_after_loc(v,h,exh,hd,vd):
    ### Изменения во временных словарях после локации
    ###  # заполняем значения для выбранного кандидата
    ### Изменения во временных словарях после локации
    ###  # заполняем значения для выбранного кандидата
    ## отнимаем абсолютные и относительные значения ресурсов
    hd[h]['free_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) 
    hd[h]['perc_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) / cpu_per_cn
    hd[h]['free_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) 
    hd[h]['perc_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[h]['vms_count'] = (hd[h]['vms_count'] + 1)
    ## обновляем список машин на ВУ
    hd[h]['vms'].append(v)
    ### # заполняем значения для ex-ВУ
    ## прибавляем ресурсы 
    hd[exh]['free_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) 
    hd[exh]['perc_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) / cpu_per_cn
    hd[exh]['free_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) 
    hd[exh]['perc_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[exh]['vms_count'] = (hd[exh]['vms_count'] - 1)
    ## обновляем список машин на ВУ
    hd[exh]['vms'].remove(v)
    vd[v]['cn_name'] = h

    
    
    ## отнимаем абсолютные и относительные значения ресурсов
    hpd_res_in_use[h]['free_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) 
    hpd_res_in_use[h]['perc_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) / cpu_per_cn
    hpd_res_in_use[h]['free_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) 
    hpd_res_in_use[h]['perc_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hpd_res_in_use[h]['vms_count'] = (hd[h]['vms_count'] + 1)
    ## обновляем список машин на ВУ
    hpd_res_in_use[h]['vms'].append(v)
    ### # заполняем значения для ex-ВУ
    ## прибавляем ресурсы 
    hpd_res_in_use[exh]['free_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) 
    hpd_res_in_use[exh]['perc_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) / cpu_per_cn
    hpd_res_in_use[exh]['free_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) 
    hpd_res_in_use[exh]['perc_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hpd_res_in_use[exh]['vms_count'] = (hd[exh]['vms_count'] - 1)
    ## обновляем список машин на ВУ
    #print(v,hpd_res_in_use[exh]['vms'])
    hpd_res_in_use[exh]['vms'].remove(v)
    vmd_in_use[v]['cn_name'] = h
    
    
    #gdictinuse[vmd[v]['groupid']]['az'][hpd[h]['az']] = gdictinuse[vmd[v]['groupid']]['az'][hpd[h]['az']] + 1 
    #gdictinuse[vmd[v]['groupid']]['hyp'][h] = gdictinuse[vmd[v]['groupid']]['hyp'][h] + 1
    #hdictinuse[h]['vms_ids'].append(v)
    #if vmd[v]['crit_rate'] == 2:
    #    hdictinuse[h]['vm_critical'] = hdictinuse[h]['vm_critical'] + 1
    #if vmd[v]['crit_rate'] == 1:
    #    hdictinuse[h]['vm_warning'] = hdictinuse[h]['vm_warning'] + 1

In [360]:
#hpd_res_in_use[test_hypervisor]

In [361]:
#find_best_hid_by_score('pg-attribute-standby-07',hpd_res_in_use)

In [362]:
#find_best_hid_by_score('pg-attribute-standby-07',hpd_res_in_use)

In [363]:
vmd, vmd_in_use = vm_dict_create(sl=vml)
hpd= hyper_dict_create(cpu_per_cn,ram_per_cn,hpl,vml)
hpd_res, hpd_res_in_use = hyper_dict_resources_create(hpd,vmd)

In [364]:
ram_list = [[i,hpd_res_in_use[i]['perc_ram']] for i in hpd_res_in_use.keys()]
cpu_list = [[i,hpd_res_in_use[i]['perc_cpu']] for i in hpd_res_in_use.keys()]
scored_list = [[i,((5 * hpd_res_in_use[i]['perc_ram'] + hpd_res_in_use[i]['perc_cpu'])/2)] for i in hpd_res_in_use.keys()]
worst_cn = sorted(ram_list, key = lambda x: float(x[1]))[:1][0][0]
sorted(ram_list, key = lambda x: float(x[1]))[:5],\
sorted(cpu_list, key = lambda x: float(x[1]))[:5],\
sorted(scored_list, key = lambda x: float(x[1]))[:5]

([['cn-e624.egron.tnx', -0.33895046325847694],
  ['cn-e13c.egron.tnx', -0.06165303003926575],
  ['cn-dce2.egron.tnx', -0.014116327201686686],
  ['cn-e8ae.egron.tnx', 0.001729240410839668],
  ['cn-e612.egron.tnx', 0.0254975918296292]],
 [['cn-e810.egron.tnx', -3.107142857142857],
  ['cn-e80c.egron.tnx', -2.9642857142857144],
  ['cn-e13c.egron.tnx', -2.7857142857142856],
  ['cn-e8b6.egron.tnx', -2.7142857142857144],
  ['cn-d720.egron.tnx', -2.642857142857143]],
 [['cn-e13c.egron.tnx', -1.5469897179553072],
  ['cn-e624.egron.tnx', -1.4723761581461923],
  ['cn-e80c.egron.tnx', -1.4183988775687841],
  ['cn-e810.egron.tnx', -1.3907926514190658],
  ['cn-e8b6.egron.tnx', -1.2735919180531263]])

In [365]:


l = start_iter_distribution(iterations=1000)
ll = copy.deepcopy(l)

In [366]:
[[i,ll[i]['vm']] for i in ll.keys()]

[[0, 'sf-bal-01'],
 [1, 'ppoz-kuvd-migration-service-09'],
 [2, 'pg-attribute-79'],
 [3, 'pg-attribute-standby-12'],
 [4, 'pg-attribute-standby-79'],
 [5, 'migr-arc-85'],
 [6, 'migr-node-005'],
 [7, 'cia-sign-bal-02'],
 [8, 'lk-rd-02'],
 [9, 'cia-core-01'],
 [10, 'lk-backend-03'],
 [11, 'isurz-xml-storage-39'],
 [12, 'isurz-api-05'],
 [13, 'ppoz-bus-ext-01'],
 [14, 'cia-idp-core-09'],
 [15, 'ppoz-gmp-process-02'],
 [16, 'isurz-xml-storage-89'],
 [17, 'ppoz-service-bal-01'],
 [18, 'cia-sign-bal-01'],
 [19, 'ppoz-gmp-bal-01'],
 [20, 'pg-mq-01'],
 [21, 'cia-core-bal-02'],
 [22, 'isurz-xml-storage-21'],
 [23, 'cia-core-bal-01'],
 [24, 'ppoz-service-notify-02'],
 [25, 'migr-gkn-oracle-01'],
 [26, 'cia-sign-10'],
 [27, 'cia-sign-05'],
 [28, 'ir-app-gi-bal-01'],
 [29, 'cite-bal-outer-01'],
 [30, 'ppoz-nsi-adapter-01'],
 [31, 'isurz-xml-storage-60'],
 [32, 'ppoz-kuvd-bal-02'],
 [33, 'pmuf-arm-sc-balancer-01'],
 [34, 'lk-rd-03'],
 [35, 'site-liferay-srv-07'],
 [36, 'mgmt-repository-01'],
 [37, 

In [367]:
#[[i,ll[i]['vm']] for i in ll.keys()]
s = [ll[i]['vm'] for i in ll.keys()if i < 131]

len(set(s))

122

In [368]:
ll[1],ll[2],ll[3],ll[4]

({'cand_hyp_from': [['cn-e624.egron.tnx', -0.48465736704035445],
   ['cn-e13c.egron.tnx', -0.09247960077318433],
   ['cn-dce2.egron.tnx', -0.0211745393739586]],
  'cand_hyp_to': [['cn-ed8e.egron.tnx', 0.4528956029794166, 316286, -22],
   ['cn-dc8e.egron.tnx', 0.2508646087768484, 172926, -64],
   ['cn-daa0.egron.tnx', 0.23898042949602508, 164734, -84],
   ['cn-dc7c.egron.tnx', 0.22115416628907575, 152446, -82],
   ['cn-e4c2.egron.tnx', 0.20926999343682384, 144254, -66]],
  'cand_vm': [[16384, 4, 'ppoz-kuvd-migration-service-09'],
   [16384, 8, 'pg-attribute-79']],
  'from': 'cn-e624.egron.tnx',
  'to': 'cn-ed8e.egron.tnx',
  'vm': 'ppoz-kuvd-migration-service-09'},
 {'cand_hyp_from': [['cn-e624.egron.tnx', -0.401468134931734],
   ['cn-e13c.egron.tnx', -0.09247960077318433],
   ['cn-dce2.egron.tnx', -0.0211745393739586]],
  'cand_hyp_to': [['cn-ed8e.egron.tnx', 0.41130098692510636, 299902, -26],
   ['cn-dc8e.egron.tnx', 0.2508646087768484, 172926, -64],
   ['cn-daa0.egron.tnx', 0.2389804

In [369]:
hpd_res[test_hypervisor],vmd['pg-attribute-standby-07']

({'az': 1,
  'cpu': 56,
  'free_cpu': -148,
  'free_ram': 17278,
  'label': 'S_CMP35',
  'migr': 0,
  'name': 'cn-d720.egron.tnx',
  'perc_cpu': -2.642857142857143,
  'perc_ram': 0.033420375635892376,
  'ram': 516990,
  'vms': ['isurz-xml-storage-21',
   'pg-attribute-73',
   'pg-attribute-standby-07',
   'pg-attribute-standby-25',
   'cia-sign-06',
   'cia-sign-08',
   'isurz-elasticsearch-07',
   'migr-arc-5',
   'migr-arc-xx',
   'migr-jst-oracle-15',
   'migr-node-022',
   'pkurp-worker-020',
   'pmo-db-fed-01',
   'ppoz-bus-command-04',
   'ppoz-bus-command-08',
   'ppoz-bus-data-08',
   'ppoz-bus-data-10',
   'ppoz-db-proc-gmp-03',
   'ppoz-db-proc-gmp-05',
   'ppoz-db-proc-pkurp-05',
   'ppoz-db-proc-popd-05',
   'ppoz-db-proc-ppoz-05',
   'ppoz-db-request-07',
   'ppoz-popd-process-04',
   'ppoz-process-core-03',
   'ppoz-process-core-05',
   'ppoz-service-external-03',
   'pg-attribute-standby-89'],
  'vms_count': 28},
 {'cn_name': 'cn-d720.egron.tnx',
  'cpu': 8,
  'crit_rate

In [370]:
#migrate_vm(hd=hpd_res_in_use,vd=vmd_in_use,v='pg-attribute-standby-07')

In [371]:
hpd_res_in_use[test_hypervisor],vmd_in_use['pg-attribute-standby-07']

({'az': 1,
  'cpu': 56,
  'free_cpu': -128,
  'free_ram': 58238,
  'label': 'S_CMP35',
  'migr': 0,
  'name': 'cn-d720.egron.tnx',
  'perc_cpu': -2.2857142857142856,
  'perc_ram': 0.11264821369852415,
  'ram': 516990,
  'vms': ['pg-attribute-73',
   'pg-attribute-standby-07',
   'pg-attribute-standby-25',
   'isurz-elasticsearch-07',
   'migr-arc-5',
   'migr-arc-xx',
   'migr-jst-oracle-15',
   'migr-node-022',
   'pmo-db-fed-01',
   'ppoz-bus-command-04',
   'ppoz-bus-command-08',
   'ppoz-bus-data-08',
   'ppoz-bus-data-10',
   'ppoz-db-proc-gmp-03',
   'ppoz-db-proc-gmp-05',
   'ppoz-db-proc-pkurp-05',
   'ppoz-db-proc-popd-05',
   'ppoz-db-proc-ppoz-05',
   'ppoz-db-request-07',
   'ppoz-popd-process-04',
   'ppoz-process-core-03',
   'ppoz-process-core-05',
   'ppoz-service-external-03',
   'pg-attribute-standby-89'],
  'vms_count': 23},
 {'cn_name': 'cn-d720.egron.tnx',
  'cpu': 8,
  'crit_rate': '1',
  'krasnoyarsk': 0,
  'name': 'pg-attribute-standby-07',
  'ram': 16384})

In [28]:
for vm in vmd.keys():
    tmp_0 = [i for i in vmd.keys() if vmd[i]['crit_rate'] == '0' and vmd[i]['krasnoyarsk'] == 1]
    tmp_2 = [i for i in vmd.keys() if vmd[i]['crit_rate'] == '2' and vmd[i]['krasnoyarsk'] == 1] 
#rmp_k = [i for i in vmd.keys() if vmd[i]['krasnoyarsk'] == 1]
    total = tmp_0 + tmp_2
#len(total), len(set(total))
#[[i, vmd[i]['crit_rate'],vmd[i]['krasnoyarsk']]for i in total]
fl = [i for i in total]
    
len(set(fl))
fl

['ppoz-db-proc-gmp-07',
 'isurz-writer-024',
 'cia-idp-core-01',
 'ppoz-kuvd-db-19',
 'ppoz-db-proc-popd-02',
 'isurz-xml-storage-right-registry-entities',
 'isurz-uploader-061',
 'cia-db-02',
 'cia-sign-bal-01',
 'isurz-xml-storage-84',
 'isurz-writer-033',
 'ppoz-db-proc-gmp-04',
 'ppoz-db-request-28',
 'ppoz-db-proc-gmp-06',
 'lk-srv-01',
 'ppoz-popd-process-03',
 'ppoz-db-proc-pkurp-04',
 'ppoz-db-proc-gmp-03',
 'ppoz-db-fs-11',
 'ppoz-db-proc-pkurp-03',
 'ppoz-pkurp-process-03',
 'ppoz-kuvd-db-16',
 'ppoz-db-proc-pkurp-05',
 'ppoz-filestorage-05',
 'pg-map-reduce-04',
 'ppoz-kuvd-db-17',
 'ppoz-process-core-03',
 'ppoz-db-proc-pkurp-06',
 'ppoz-pkurp-process-04',
 'ppoz-db-fs-23',
 'ppoz-db-proc-popd-03',
 'ppoz-db-request-24',
 'ppoz-pkurp-process-05',
 'ppoz-gmp-services-04',
 'ppoz-bus-ext-03',
 'ppoz-db-fs-28',
 'ppoz-db-proc-popd-04',
 'ppoz-popd-process-04',
 'ppoz-db-fs-20',
 'ppoz-kuvd-db-11',
 'ppoz-db-request-15',
 'ppoz-db-fs-09',
 'ppoz-bus-ext-02',
 'ppoz-bus-ext-04',

In [372]:
#scoring_list = [[i, score_for_test[i]['score'], score_for_test[i]['ram']] for i in score_for_test.keys() ]

In [373]:
#sorted(scorint_list, key = lambda x: float(x[1]), reverse=True)[:1]

In [374]:
#ram_test = [[i,hpd_res_in_use[i]['free_ram']]for i in hpd_res_in_use.keys()]


In [375]:
#cn_test = sorted(ram_test, key = lambda x: int(x[1]))
#w_cn = cn_test[:1][0][0]
#hpd_res_in_use[w_cn]['vms']
#mean_memory =  mean([vmd[j]['ram'] for j in hpd_res_in_use[w_cn]['vms']])
#cand_list = [[vmd[j]['ram'], j ]for j in hpd_res_in_use[w_cn]['vms'] if vmd[j]['ram'] > 0]
#cand = sorted(cand_list)[:1]
#cand[0][1]

In [376]:
#cn_test
#mean([vmd[j]['ram'] for j in hpd_res_in_use[cn_test]['vms']])

In [377]:
l = [hpd_res[h]['perc_ram'] for h in hpd_res.keys()]
m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]



print("Всего машин:", len(vml))
print("Всего машин:", len(vmd.keys()))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
print("Всего ВУ:", len(hpl))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))
print(np.std(l), np.std(m))
#print(np.std(l, ddof = 1), np.std(m, ddof = 1))

Всего машин: 1654
Всего машин: 1652
Необходимо машинам vCPU : 12118 , RAM : 34728472 MB
На одном ВУ CPU : 56 , RAM : 516990 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 39291240 MB
Суммарная подписка по CPU : 2.8472744360902253 , RAM : 0.8838731483149934
Забитость ВУ по RAM
Самый забитый : 1.338950463258477
Наименее забитый : 0.3802936227006325
Средняя загруженность : 0.8849156198684491

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 4.107142857142858
Наименее забитый : 1.3571428571428572
Средняя загруженность : 2.8500939849624056
0.110834119475 0.510069777641


In [378]:
vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hpd_res_in_use,vmd_in_use)

In [379]:
l = [hpd_res_in_use[h]['perc_ram'] for h in hpd_res_in_use.keys()]
m = [hpd_res_in_use[h]['perc_cpu'] for h in hpd_res_in_use.keys()]



print("Всего машин:", len(vmd_in_use.keys()))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
print("Всего ВУ:", len(hpl))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))
print(np.std(l), np.std(m))
#print(np.std(l, ddof = 1), np.std(m, ddof = 1))

Всего машин: 1652
Необходимо машинам vCPU : 12118 , RAM : 34728472 MB
На одном ВУ CPU : 56 , RAM : 516990 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 39291240 MB
Суммарная подписка по CPU : 2.8472744360902253 , RAM : 0.8838731483149934
Забитость ВУ по RAM
Самый забитый : 0.9190429215265286
Наименее забитый : 0.8715062186889495
Средняя загруженность : 0.8834561596936112

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 3.9285714285714284
Наименее забитый : 0.5714285714285714
Средняя загруженность : 2.840225563909775
0.0100054786639 0.511530372352


In [380]:
temp_list_hp =  [h for h in hpd.keys() if hpd[h]['migr'] == 0 and hpd[h]['az'] == 3 ]
#temp_list_hp
temp_list_vm = [hpd[h]['vms'] for h in temp_list_hp]
ola = list(itertools.chain.from_iterable(temp_list_vm))

ola1 = [vmd[i]['crit_rate'] for i in ola]
ola2= [[i,vmd[i]['crit_rate']]for i in vmd.keys()]
sorted(ola2,key = lambda x: x[1], reverse=True)

[['pkurp-app-035', 'E'],
 ['migr-gkn-oracle-47', 'E'],
 ['ppoz-smev-service-02', 'E'],
 ['ppoz-popd-process-06', 'E'],
 ['popd-geocdn-01', 'E'],
 ['ir-bd-01', 'E'],
 ['isurz-xml-storage-56', 'E'],
 ['pg-attribute-standby-83', 'E'],
 ['isurz-uploader-007', 'E'],
 ['ppoz-db-request-17', 'E'],
 ['pkurp-worker-012', 'E'],
 ['Isurz-api-01', 'E'],
 ['ppoz-kuvd-db-13', 'E'],
 ['pkurp-worker-006', 'E'],
 ['migr-grp-oracle-89', 'E'],
 ['ppoz-db-proc-popd-01', 'E'],
 ['migr-gkn-oracle-24', 'E'],
 ['isurz-xml-storage-04', 'E'],
 ['site-solr-03', 'E'],
 ['migr-gkn-oracle-31', 'E'],
 ['migr-arc-64', 'E'],
 ['popd-bpd-balancer-01', 'E'],
 ['migr-jst-oracle-74', 'E'],
 ['migr-node-055', 'E'],
 ['migr-grp-oracle-36', 'E'],
 ['isurz-writer-031', 'E'],
 ['migr-node-166', 'E'],
 ['migr-gkn-oracle-39', 'E'],
 ['ppoz-db-proc-ppoz-02', 'E'],
 ['migr-node-017', 'E'],
 ['migr-gkn-oracle-89', 'E'],
 ['pmuf-arm-sc-balancer-01', 'E'],
 ['ppoz-kuvd-db-bkp-08', 'E'],
 ['pkurp-worker-002', 'E'],
 ['ppoz-db-fs-bkp-0

In [381]:
azc_d={}
for i in range(1,4):
   # print('Зона доступности ',i,' *без ВУ под миграцию')
    temp_list_hp =  [h for h in hpd.keys() if hpd[h]['migr'] == 0 and hpd[h]['az'] == i ]
    #print(temp_list_hp)
    for h in temp_list_hp:
        azc_d[h]={}
        temp_list_vm = hpd[h]['vms']
        crit_2_vm = [v for v in temp_list_vm if vmd[v]['crit_rate'] == '2']
        crit_1_vm = [v for v in temp_list_vm if vmd[v]['crit_rate'] == '1']
        crit_0_vm = [v for v in temp_list_vm if vmd[v]['crit_rate'] == '0']
        crit_k_vm = [v for v in temp_list_vm if vmd[v]['krasnoyarsk'] == 1]
        azc_d[h]['vm0']=len(crit_0_vm)
        azc_d[h]['vm2']=len(crit_2_vm)
        azc_d[h]['vm1']=len(crit_1_vm)
        azc_d[h]['az']=i
        azc_d[h]['krs']=len(crit_k_vm)
       # print('\tВУ',h,'ВМ 0 уровня:', len(crit_0_vm),'ВМ 2 уровня:',len(crit_2_vm))

In [448]:
#hpd
for i in range(1,4):
    hl = [h for h in azc_d.keys() if azc_d[h]['az'] == i]
    
    
    #print()
    #tl = [[h,azc_d[h]['vm0'],azc_d[h]['vm2']] for h in hl if azc_d[h]['vm0'] != 0 or azc_d[h]['vm2'] != 0]
    tl = [[h,azc_d[h]['vm0'],azc_d[h]['vm2'],azc_d[h]['vm1'], azc_d[h]['krs']] for h in hl if azc_d[h]['vm0'] == 0 and azc_d[h]['vm2'] == 0]
    print('Зона доступности ',i,'всего ВУ:',len(hl) )
    print('\tвсего ВУ без ВМ 2 и 0 уровня:',len(tl) )
    print('')
    [print('\t',l[0],l[1],l[2],l[3],l[4]) for l in tl ]
    print('')

Зона доступности  1 всего ВУ: 37
	всего ВУ без ВМ 2 и 0 уровня: 5

	 cn-db74.egron.tnx 0 0 5 5
	 cn-e89a.egron.tnx 0 0 6 11
	 cn-dcec.egron.tnx 0 0 3 8
	 cn-e0f6.egron.tnx 0 0 6 3
	 cn-e624.egron.tnx 0 0 4 1

Зона доступности  2 всего ВУ: 13
	всего ВУ без ВМ 2 и 0 уровня: 7

	 cn-db28.egron.tnx 0 0 1 8
	 cn-dc8e.egron.tnx 0 0 3 4
	 cn-db38.egron.tnx 0 0 2 3
	 cn-dce2.egron.tnx 0 0 2 1
	 cn-db3e.egron.tnx 0 0 1 5
	 cn-e4c2.egron.tnx 0 0 1 8
	 cn-e13a.egron.tnx 0 0 2 7

Зона доступности  3 всего ВУ: 15
	всего ВУ без ВМ 2 и 0 уровня: 9

	 cn-dce8.egron.tnx 0 0 10 5
	 cn-dca6.egron.tnx 0 0 1 9
	 cn-ed8e.egron.tnx 0 0 7 2
	 cn-dca4.egron.tnx 0 0 2 5
	 cn-e810.egron.tnx 0 0 1 0
	 cn-ed8c.egron.tnx 0 0 1 2
	 cn-dc8c.egron.tnx 0 0 2 7
	 cn-dca2.egron.tnx 0 0 5 3
	 cn-da22.egron.tnx 0 0 1 4



In [449]:
hpd['cn-e810.egron.tnx']

{'az': 3,
 'cpu': 56,
 'label': 'S_CMP76',
 'migr': 0,
 'name': 'cn-e810.egron.tnx',
 'ram': 516990,
 'vms': ['isurz-xml-storage-43',
  'ir-app-gi-01',
  'ir-app-gi-02',
  'ir-app-gi-03',
  'ir-app-gi-04',
  'ir-app-lk-01',
  'ir-app-lk-02',
  'ir-app-lk-04',
  'ir-app-out-04',
  'migr-init-01',
  'pkurp-app-037',
  'pmuf-zabbix-db-01',
  'popd-api-migr-01',
  'popd-api-migr-02',
  'ppoz-db-fs-bkp-01',
  'ppoz-kuvd-migration-service-83',
  'zbx_bcp'],
 'vms_count': 17}

In [450]:
oola1 = [[i,azc_d[i]['az'],azc_d[i]['vm0'],azc_d[i]['vm2'],azc_d[i]['vm1'],azc_d[i]['krs'],hpd_res[i]['free_ram'],len(hpd_res[i]['vms'])]\
        for i in azc_d.keys()\
        if azc_d[i]['az'] == 1 and (azc_d[i]['vm0'] == 0 and azc_d[i]['vm2'] == 0)]

In [451]:
sorted(oola1, key =operator.itemgetter(4,5))

[['cn-dcec.egron.tnx', 1, 0, 0, 3, 8, 17278, 25],
 ['cn-e624.egron.tnx', 1, 0, 0, 4, 1, -175234, 13],
 ['cn-db74.egron.tnx', 1, 0, 0, 5, 5, 16062, 27],
 ['cn-e0f6.egron.tnx', 1, 0, 0, 6, 3, 86910, 21],
 ['cn-e89a.egron.tnx', 1, 0, 0, 6, 11, 41854, 26]]

In [452]:
oola2 = [[i,azc_d[i]['az'],azc_d[i]['vm0'],azc_d[i]['vm2'],azc_d[i]['vm1'],azc_d[i]['krs'],hpd_res[i]['free_ram'],len(hpd_res[i]['vms'])]\
        for i in azc_d.keys()\
        if azc_d[i]['az'] == 2 and (azc_d[i]['vm0'] == 0 and azc_d[i]['vm2'] == 0)]

In [453]:
sorted(oola2, key =operator.itemgetter(4,5))

[['cn-db3e.egron.tnx', 2, 0, 0, 1, 5, 74622, 21],
 ['cn-db28.egron.tnx', 2, 0, 0, 1, 8, 62334, 25],
 ['cn-e4c2.egron.tnx', 2, 0, 0, 1, 8, 144254, 22],
 ['cn-dce2.egron.tnx', 2, 0, 0, 2, 1, -7298, 22],
 ['cn-db38.egron.tnx', 2, 0, 0, 2, 3, 111486, 22],
 ['cn-e13a.egron.tnx', 2, 0, 0, 2, 7, 103294, 19],
 ['cn-dc8e.egron.tnx', 2, 0, 0, 3, 4, 172926, 21]]

In [454]:
oola3 = [[i,azc_d[i]['az'],azc_d[i]['vm0'],azc_d[i]['vm2'],azc_d[i]['vm1'],azc_d[i]['krs'],hpd_res[i]['free_ram'],len(hpd_res[i]['vms'])]\
        for i in azc_d.keys()\
        if azc_d[i]['az'] == 3 and (azc_d[i]['vm0'] == 0 and azc_d[i]['vm2'] == 0)]

In [430]:
sorted(oola3, key =operator.itemgetter(4,5))

[['cn-e810.egron.tnx', 3, 0, 0, 1, 0, 33662, 17],
 ['cn-ed8c.egron.tnx', 3, 0, 0, 1, 2, 103294, 24],
 ['cn-da22.egron.tnx', 3, 0, 0, 1, 4, 107390, 20],
 ['cn-dca6.egron.tnx', 3, 0, 0, 1, 9, 33662, 27],
 ['cn-dca4.egron.tnx', 3, 0, 0, 2, 5, 74622, 25],
 ['cn-dc8c.egron.tnx', 3, 0, 0, 2, 7, 82814, 19],
 ['cn-dca2.egron.tnx', 3, 0, 0, 5, 3, 99198, 22],
 ['cn-ed8e.egron.tnx', 3, 0, 0, 7, 2, 320382, 9],
 ['cn-dce8.egron.tnx', 3, 0, 0, 10, 5, 136062, 11]]

In [436]:
hpd_res['cn-ed8e.egron.tnx'], hpd_res_in_use['cn-ed8e.egron.tnx']

({'az': 3,
  'cpu': 56,
  'free_cpu': -20,
  'free_ram': 320382,
  'label': 'S_CMP29',
  'migr': 0,
  'name': 'cn-ed8e.egron.tnx',
  'perc_cpu': -0.35714285714285715,
  'perc_ram': 0.6197063772993675,
  'ram': 516990,
  'vms': ['pg-master',
   'ppoz-kuvd-migration-service-50',
   'pg-attribute-61',
   'pg-attribute-64',
   'pg-attribute-67',
   'pg-attribute-69',
   'pg-attribute-71',
   'pg-attribute-76',
   'isurz-writer-060'],
  'vms_count': 9},
 {'az': 3,
  'cpu': 56,
  'free_cpu': -126,
  'free_ram': 107390,
  'label': 'S_CMP29',
  'migr': 0,
  'name': 'cn-ed8e.egron.tnx',
  'perc_cpu': -2.25,
  'perc_ram': 0.20772161937368228,
  'ram': 516990,
  'vms': ['pg-master',
   'ppoz-kuvd-migration-service-50',
   'pg-attribute-61',
   'pg-attribute-64',
   'pg-attribute-67',
   'pg-attribute-69',
   'pg-attribute-71',
   'pg-attribute-76',
   'isurz-writer-060',
   'zbx_bcp',
   'isurz-xml-storage-43',
   'ir-app-gi-bal-01',
   'pkurp-app-037',
   'ppoz-kuvd-migration-service-83',
   'mi

In [385]:
#rl = [0.0001,0.001,0.001,0.1,1,10,100,10000]
#cl = [0.0001,0.001,0.001,0.1,1,10,100,10000]
rl = [1]
cl = [1]

In [386]:
rd={}
c=1
for i in rl:
    for j in cl:
        rd[c]={}
        vmd, vmd_in_use = vm_dict_create(sl=vml)
        hpd= hyper_dict_create(cpu_per_cn,ram_per_cn,hpl,vml)
        hpd_res, hpd_res_in_use = hyper_dict_resources_create(hpd,vmd)
        start_iter_distribution(cpu_coef=j,ram_coef=i,iterations=135)
        l = [hpd_res_in_use[h]['perc_ram'] for h in hpd_res_in_use.keys()]
        n = [hpd_res_in_use[h]['perc_cpu'] for h in hpd_res_in_use.keys()]
        rd[c]['rc']=i
        rd[c]['cc']=j
        
        rd[c]['rmax']=1-min(l)
        rd[c]['rmin']=1-max(l)
        rd[c]['ram_std']=np.std(l)
        rd[c]['cmax']=1-min(n)
        rd[c]['cmin']=1-max(n)
        rd[c]['cpu_std']=np.std(n)
        c=c+1
        

In [387]:
#rd

In [388]:
#std_list = [[rd[i]['cc'],rd[i]['rc'],rd[i]['ram_std'],rd[i]['rmax'],rd[i]['rmin'],\
#             rd[i]['cpu_std'],rd[i]['cmax'],rd[i]['cmin']] for i in rd.keys()]
#std_list = [[rd[i]['cc'],rd[i]['rc'],rd[i]['ram_std'],rd[i]['cpu_std']] for i in rd.keys()]

In [389]:
#0.00847 - 145
#0.00838 - 140
#0.0086615711224086369 - 135
#sorted(std_list, key=lambda x: float(x[2]))